# Drive characteristics using `strym`

In this notebook, we will generate characteristics of a particular drive. Please ensure strym is installed correctly

## Installing `strym`
First, we are required to install `strym`. For that, we will just use github devtools functionality.
Even if you have installed `strym` previously, you should reinstall for latest version of the package.

In [11]:
# Installing strym using pip from git
#!pip install gmplot gmaps googlemaps dotenv
!pip install python-dotenv selenium m2r mkdocs pymdown-extensions sphinx-bootstrap-theme sphinx-markdown-parser webdriver-manager ytsphinx
#!pip install --upgrade --no-deps --force-reinstall git+https://github.com/jmscslgroup/strym
!pip install --upgrade --no-deps --force-reinstall git+https://github.com/jmscslgroup/strym

  Using cached https://files.pythonhosted.org/packages/bb/e9/da0c2903cf4ef7649acd2d1400680921ced26c1ae4457b7988f8d693d4cb/mkdocs-1.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/fc/7f/fc169385d5ff6ab000597c9ef81e798e9b20ac267f33426dcc89d9f40306/pymdown_extensions-7.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/67/b9/cb6b7696617e5f59d7acd78ca48e494007b702367e77099eeaa13f0bb5be/sphinx_markdown_parser-0.2.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/c2/f6/bd9d73991fd8d1b488aefa4b55711af670e938a9b8549ffd8ecc42780069/webdriver_manager-2.3.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/1b/39/6531ec0e2e6702a46cb3f28887f840d80b8136a51688969ecab1d9b05932/ytsphinx-1.0.dev20200401-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/12/4d/30cfe74402d2e962d66d35da29bf8850b0557b559ce84d09967c8ade859e/livereload-2.6.1-py2.py3-none-any.whl
  Using cached ht

## Importing packages
Import required packages

In [12]:
from strym import strymread
import strym
import math
import time
import matplotlib.pyplot as plt
import numpy as np
import scipy.integrate as integrate

Loading BokehJS ...

## Specify Data Location

In [13]:
# datafolder = "/Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/2020_03_03/"
# import glob
# csvlist = glob.glob(datafolder+"*.csv")
# datafolder = "/Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/2020-03-05/"
# # print(csvlist)
# for f in glob.glob(datafolder+"*.csv"):
#     csvlist.append(f)
# print(csvlist)

In [66]:
import glob
parentfolder = "/Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/testData/"
folderlist = glob.glob(parentfolder+"*")
print(folderlist)
csvlist = []
for datafolder in folderlist:
#     datafolder = "/Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/PandaData/2020_03_03/"
    csvlisttmp = glob.glob(datafolder+"/*CAN_Messages.csv")
    for f in csvlisttmp:
        csvlist.append(f)
#print(csvlist)

['/Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/testData/truncatedfile-test']


In [67]:
num_of_files = len(csvlist)
print("Total number of datafiles in {} is {}.".format(parentfolder, num_of_files))

Total number of datafiles in /Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/testData/ is 6.


## Analysis
### 1. CSV file containing all messages
In this section, we will extract a subset of the CSV-formatted CAN Data in order to produce summary information of a specific drive.

In [68]:
import sys
dbcfile = '../examples/newToyotacode.dbc'
metadata = []
for i in range(num_of_files):
    try:
        print(f'Reading {csvlist[i]}')
        val = 1
        if val==0:
            r0 = strymread(csvfile=csvlist[i], dbcfile=dbcfile)
            drive = r0.driving_characteristics()
        else:
            r0 = strymread(csvfile=csvlist[i], dbcfile=dbcfile)
            duration1 = r0.dataframe['Time'][len(r0.dataframe)-1] - r0.dataframe['Time'][0]
            duration_str = f'  Duration of this drive is {duration1} seconds ({math.trunc(int(duration1*1000)/(1000*60))} minutes {math.trunc(duration1 % 60)} seconds).'
            start_str = f'  Starting date/time of the drive is {time.ctime(r0.dataframe["Time"][0])}'

            # get the speed timeseries information from the data file
            speed_ts = r0.speed()
            # turn the timeseries into a python array for integration
            # transform from km/hr by multiplying 1000m/1km and 1 hr/3600s to get m/s
            speed_ar = np.array(speed_ts['Message'])*1000/3600
        #     speed_ar[0:-1]*1000/3600
            # find the difference of the time values
        #     dt = np.diff(np.array(speed_ts['Time']))
            dt = np.diff(speed_ts['Time'])
            # trapezoidal integration, divide by 1000 to get total km (rather than m)
            # Commented out: this produces negative values that don't make sense
        #     km_ts = np.trapz(y=speed_ar[0:-1],x=dt)/1000
            km_ts = np.trapz(y=speed_ar,x=np.array(speed_ts['Time']))/1000
            # commented out: this produces incorrect values that seem to be off by a factor of around 2, depending
            km_dx = np.trapz(y=speed_ar,dx=0.02)/1000
            # need to convert km/hr to km/s to get km later when integrating
            distance_str = f'  The trip was {km_ts} km ({km_ts*3.1/5} miles)'
        #     distance_str_dx f'  The dx version is {km_dx} km ({km_dx*3.1/5} miles)'

            #speed_ts['Time']
            #f'Total miles driven is {km_ts}'
        #     print(duration_str)
        #     print(start_str)
        #     print(distance_str)
        #     print('')
            drive = { 'filename': csvlist[i], 'distance_km': km_ts, 'distance_miles': km_ts*3.1/5, 'duration_s': duration1, 
                     'date': time.ctime(r0.dataframe['Time'][0]) }
        metadata.append( drive )
    except:
        print(f'Unable to process ', csvlist[i], ', exception=', sys.exc_info() )
print(metadata)

Reading /Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/testData/truncatedfile-test/Fail2-2020-04-19-12-02-08_2T3Y1RFV8KC014025_CAN_Messages.csv
Unable to process  /Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/testData/truncatedfile-test/Fail2-2020-04-19-12-02-08_2T3Y1RFV8KC014025_CAN_Messages.csv , exception= (<class 'ValueError'>, ValueError('Cannot convert non-finite values (NA or inf) to integer'), <traceback object at 0x1b8822f00>)
Reading /Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/testData/truncatedfile-test/Pass1-2020-04-19-12-02-08_2T3Y1RFV8KC014025_CAN_Messages.csv
Reading /Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/testData/truncatedfile-test/Fail0-2020-04-19-12-02-08_2T3Y1RFV8KC014025_CAN_Messages.csv
Unable to process  /Users/sprinkle/work/data/cyverse/rahulbhadani/JmscslgroupData/testData/truncatedfile-test/Fail0-2020-04-19-12-02-08_2T3Y1RFV8KC014025_CAN_Messages.csv , exception= (<class 'ValueError'>, 

In [49]:
# how did we do?
dist=0
for d in metadata:
    dist = dist + d['distance_miles']
print(dist)

2.297263271267467


In [ ]:
metadata